<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/ML_Spark_Regressao_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 43.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=66235149ddb928b2886d6723d5dadd6fdec22a6303ede100444edb920359a13f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('Forest_spark').getOrCreate()

In [4]:
carros = spark.read.csv('Carros.csv', header=True, sep=';',inferSchema=True)
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [5]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import RFormula

In [6]:
Rformula = RFormula(formula='HP ~ Consumo + Cilindros + Cilindradas', featuresCol='independente', labelCol='dependente')
carrosrf = Rformula.fit(carros).transform(carros)
carrosrf.select('independente','dependente').show(truncate=False)

+------------------+----------+
|independente      |dependente|
+------------------+----------+
|[21.0,6.0,160.0]  |110.0     |
|[21.0,6.0,160.0]  |110.0     |
|[228.0,4.0,108.0] |93.0      |
|[214.0,6.0,258.0] |110.0     |
|[187.0,8.0,360.0] |175.0     |
|[181.0,6.0,225.0] |105.0     |
|[143.0,8.0,360.0] |245.0     |
|[244.0,4.0,1467.0]|62.0      |
|[228.0,4.0,1408.0]|95.0      |
|[192.0,6.0,1676.0]|123.0     |
|[178.0,6.0,1676.0]|123.0     |
|[164.0,8.0,2758.0]|180.0     |
|[173.0,8.0,2758.0]|180.0     |
|[152.0,8.0,2758.0]|180.0     |
|[104.0,8.0,472.0] |205.0     |
|[104.0,8.0,460.0] |215.0     |
|[147.0,8.0,440.0] |230.0     |
|[324.0,4.0,787.0] |66.0      |
|[304.0,4.0,757.0] |52.0      |
|[339.0,4.0,711.0] |65.0      |
+------------------+----------+
only showing top 20 rows



In [7]:
from pyspark.ml.feature import Normalizer

In [8]:
normalizer = Normalizer(inputCol='independente',outputCol='independente_norm',p=1.0)
carros_norm = normalizer.transform(carrosrf)

In [9]:
carros_norm.select('independente','dependente','independente_norm').show(truncate=False)

+------------------+----------+--------------------------------------------------------------+
|independente      |dependente|independente_norm                                             |
+------------------+----------+--------------------------------------------------------------+
|[21.0,6.0,160.0]  |110.0     |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|[21.0,6.0,160.0]  |110.0     |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|[228.0,4.0,108.0] |93.0      |[0.6705882352941176,0.011764705882352941,0.3176470588235294]  |
|[214.0,6.0,258.0] |110.0     |[0.4476987447698745,0.012552301255230125,0.5397489539748954]  |
|[187.0,8.0,360.0] |175.0     |[0.33693693693693694,0.014414414414414415,0.6486486486486487] |
|[181.0,6.0,225.0] |105.0     |[0.4393203883495146,0.014563106796116505,0.5461165048543689]  |
|[143.0,8.0,360.0] |245.0     |[0.27984344422700586,0.015655577299412915,0.7045009784735812] |
|[244.0,4.0,1467.0]|62.0      |[0.1422740524781341

In [20]:
carro_treino, carro_teste =  carros_norm.randomSplit([0.7,0.3], seed=1)

In [21]:
carro_treino.count() , carro_teste.count()

(24, 8)

In [22]:
rf = RandomForestRegressor(featuresCol='independente_norm', labelCol='dependente',maxDepth=10,numTrees=500,seed=20)

In [24]:
modelo = rf.fit(carro_treino)

In [26]:
previsao = modelo.transform(carro_teste)
previsao.select('dependente','prediction').show()

+----------+------------------+
|dependente|        prediction|
+----------+------------------+
|     215.0|            175.44|
|     205.0|            175.44|
|     180.0|           158.596|
|     180.0|           158.596|
|     123.0|           125.568|
|     105.0| 167.3863333333333|
|     175.0|169.75033333333332|
|     113.0|           114.878|
+----------+------------------+



In [27]:
from pyspark.ml.evaluation import RegressionEvaluator

In [29]:
avaliacao = RegressionEvaluator(predictionCol='prediction',labelCol='dependente', metricName='rmse')

In [31]:
rmse = avaliacao.evaluate(previsao)
print(rmse) # mais perto de 0 melhor

30.176117889170413


com Random Forest a performance foi melhor do que com a regressao linear